In [23]:
#IMPLEMENTING AN AI ALGORITHM TO SOLVE A 9X9 SUDOKU GAME 
#This is a CSP - Constraint Satisfaction Problem

In [1]:
from copy import deepcopy

Grid = [['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.']]

#Elements allowed in the game
elements = ['1', '2', '3', '4', '5', '6', '7', '8', '9']

In [1]:
#IMPLEMENTING THE THREE CONSTRAINTS OF THE SUDOKU GAME:

In [2]:
#Getting the rows to be filled
def get_rows(new_Grid):
    rows = []
    for r in range (0,9):
        row = {}
        for c in range(0,9):
            row[(r,c)] = new_Grid[r][c]   
        rows.append(row)
    return rows

In [4]:
#Getting the columns to be filled
def get_cols(new_Grid):
    cols = []
    for c in range(0,9):        #now we are looping over columns before the rows
        col = {}
        for r in range(0,9):
            col[(r,c)] = new_Grid[r][c]
            cols.append(col)
    return cols

In [5]:
#Implementing the fourth constraint of the Sudoku Game

In [25]:
## This cell is only for clarification, to check that we are going t
cols = get_cols(Grid)
for c in cols:
    print(c)

{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
{(0, 0): '.', (1, 0): '.', (2, 0): '.', (3, 0): '.', (4,

In [26]:
#To ge the squares, we need first to specify the start and end of each small square in the rows and columns. 
square_indx = [[(0,2), (0,2)],   
               [(0,2), (3,5)],  
               [(0,2), (6,8)],
               [(3,5), (0,2)],
               [(3,5), (3,5)],   
               [(3,5), (6,8)], 
               [(6,8), (0,2)],
               [(6,8), (3,5)],
               [(6,8), (6,8)]]

def get_squares(new_Grid):
    squares = []
    for sq in range(0,9):
        square = {}
        #Looping over rows and columns 
        for r in square_indx[sq][0] :
            for c in square_indx[sq][1]  :
                square[(r,c)] = new_Grid[r][c]  
        squares.append(square)

    return squares

In [27]:
## This cell is only for clarification and it is not part of the original code
squares = get_squares(Grid)
for sq in squares:
    print(sq)

{(0, 0): '.', (0, 2): '2', (2, 0): '.', (2, 2): '.'}
{(0, 3): '3', (0, 5): '.', (2, 3): '.', (2, 5): '.'}
{(0, 6): '.', (0, 8): '.', (2, 6): '.', (2, 8): '.'}
{(3, 0): '.', (3, 2): '.', (5, 0): '.', (5, 2): '.'}
{(3, 3): '.', (3, 5): '.', (5, 3): '.', (5, 5): '.'}
{(3, 6): '.', (3, 8): '.', (5, 6): '.', (5, 8): '.'}
{(6, 0): '.', (6, 2): '.', (8, 0): '.', (8, 2): '.'}
{(6, 3): '.', (6, 5): '.', (8, 3): '.', (8, 5): '.'}
{(6, 6): '.', (6, 8): '.', (8, 6): '.', (8, 8): '.'}


In [28]:
def get_all_related_cells(new_Grid):
    rows = get_rows(new_Grid)
    cols = get_cols(new_Grid)
    squares = get_squares(new_Grid)

    all_vec = squares + rows + cols
    return all_vec

In [29]:
def get_new_r_c(r, c):
    #3 is the last cell
    if c==8:
        if r==8:
            new_r = r
            new_c = c
        else:
            new_r = r+1
            new_c = 0
            #we started the new row but at the first column. 
    else:
        new_r = r
        new_c = c+1
    return new_r, new_c

In [30]:
#How to obtain the candidate solution that don't violate the four constraints of the game:

def get_legal_for_cell(cell_r, cell_c, new_Grid):
    #Checking if the cell has value
    if new_Grid[cell_r][cell_c]!='.':
        return [None],[None],[None]
    #We write 3x none since this function has 3 containers
    
    map = {}
    all_vec = get_all_related_cells(new_Grid)
    for a in all_vec:
        if (cell_r, cell_c) in a.keys():
            map.update(a)          # no duplicates

    exist = []
    for m in map:                  # get key from keys
        if not map[m]=='.':
            exist.append(map[m])

    rest = list(set(elements) - set(exist))
    return rest, exist, map

In [31]:
## This cell is only for clarification and it is not part of the original code
r = 0
c = 0
legal_for_cell, exist, map = get_legal_for_cell(r, c, Grid)
print(map)
print(exist)
print(legal_for_cell)

{(0, 0): '.', (0, 2): '2', (2, 0): '.', (2, 2): '.', (0, 1): '.', (0, 3): '3', (0, 4): '.', (0, 5): '.', (0, 6): '.', (0, 7): '.', (0, 8): '.', (1, 0): '.', (3, 0): '.', (4, 0): '.', (5, 0): '.', (6, 0): '.', (7, 0): '.', (8, 0): '.'}
['2', '3']
['9', '6', '4', '7', '5', '1', '8']


In [32]:
def is_complete(new_Grid):
    grid_complete = True
    for r in new_Grid:
        grid_complete = grid_complete and not ('.' in r)
    return grid_complete

def print_grid(new_Grid):
    for item in new_Grid:
        print(item)
    print()

In [34]:
#Just checking the grid:

print_grid(Grid)

['.', '.', '2', '3', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']
['.', '.', '.', '.', '.', '.', '.', '.', '.']



In [36]:
#Implementing backtracking algorithm
def solve_step_in_sudoku(last_Grid, r, c):
    if is_complete(last_Grid):
        print('Complete:')
        print_grid(last_Grid)
        return 0
    
    legal_for_cell,_ ,_ = get_legal_for_cell(r, c, last_Grid)

    for item in legal_for_cell:
        new_Grid = deepcopy(last_Grid)
        if last_Grid[r][c]=='.':
            new_Grid[r][c] = item
        new_r, new_c = get_new_r_c(r, c)
        
        solve_step_in_sudoku(new_Grid, new_r, new_c)

print('Incomplete:')
print_grid(Grid)
solve_step_in_sudoku(Grid, 0, 0)

6', '2', '9', '5', '3']

Complete:
['9', '6', '2', '3', '4', '7', '5', '1', '8']
['2', '9', '3', '6', '7', '5', '1', '8', '4']
['3', '2', '6', '9', '5', '8', '7', '4', '1']
['6', '3', '9', '2', '1', '4', '8', '7', '5']
['4', '7', '5', '1', '8', '9', '2', '3', '6']
['7', '1', '8', '5', '2', '3', '4', '6', '9']
['5', '4', '7', '8', '6', '1', '9', '2', '3']
['1', '8', '4', '7', '9', '6', '3', '5', '2']
['8', '5', '1', '4', '3', '2', '6', '9', '7']

Complete:
['9', '6', '2', '3', '4', '7', '5', '1', '8']
['2', '9', '3', '6', '7', '5', '1', '8', '4']
['3', '2', '6', '9', '5', '8', '7', '4', '1']
['6', '3', '9', '2', '1', '4', '8', '7', '5']
['4', '7', '5', '1', '8', '9', '2', '3', '6']
['7', '1', '8', '5', '2', '3', '4', '6', '9']
['5', '4', '7', '8', '6', '1', '3', '9', '2']
['1', '8', '4', '7', '9', '2', '6', '5', '3']
['8', '5', '1', '4', '3', '6', '9', '2', '7']

Complete:
['9', '6', '2', '3', '4', '7', '5', '1', '8']
['2', '9', '3', '6', '7', '5', '1', '8', '4']
['3', '2', '6', '9', '5

KeyboardInterrupt: 